<a href="https://colab.research.google.com/github/Sushila-Phogat/CE888/blob/master/Lab8/Flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "sushilakumari" # username from the json file
os.environ['KAGGLE_KEY'] = "5e4b53595f841ece8b9f6d42ff83cbd2" # key from the json file
!kaggle datasets download -d alxmamaev/flowers-recognition # api copied from kaggle

100% 449M/450M [00:11<00:00, 57.5MB/s]
100% 450M/450M [00:11<00:00, 41.4MB/s]


In [6]:
!git clone https://github.com/sagihaider/CE888_2020.git

fatal: destination path 'CE888_2020' already exists and is not an empty directory.


In [2]:
# Upload data
from zipfile import ZipFile
file_name = "/content/flowers-recognition.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [3]:
!pip install tensorflow==1.3.0
!pip install keras==2.0.7

     |████████████████████████████████| 43.6MB 93kB/s 
     |████████████████████████████████| 1.6MB 32.9MB/s 
     |████████████████████████████████| 890kB 45.3MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=12ef8aaa9bf736925ca40494c2c9b1576c7ce672e746fb066c43845a9cdac68b
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.3
    Uninstalling bleach-3.1.3:
      Successfully uninstalled bleach-3.1.3
  Found existing installation: tensorflow 2.2.0rc1
    Uninstalling tensorflow-2.2.0rc1:
      Successfully uninstalled tensorflow-2.2.0rc1
     |████████████████████████████████| 276kB 1.4MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.7 which is incompat

In [7]:
import numpy as np
import os
import time
from CE888_2020.Lab_8.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:461: FutureWarn

In [8]:
#%% Loading the training data
PATH = os.getcwd()
print(PATH)
# Define data path
data_path = PATH + '/flowers'
data_dir_list = os.listdir(data_path)
print(data_dir_list)


/content
['dandelion', 'daisy', 'sunflower', 'rose', 'flowers', 'tulip']


In [10]:
img_data_list=[] # created an empty list for images to be loaded

for dataset in data_dir_list:
	img_list=os.listdir(data_path+'/'+ dataset)
	print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
	print(img_list)
	for img in img_list:
		img_path = data_path + '/'+ dataset + '/'+ img
		img = image.load_img(img_path, target_size=(224, 224))
		x = image.img_to_array(img)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
#		x = x/255
		# print('Input image shape:', x.shape)
		img_data_list.append(x)

#%% 
img_data = np.array(img_data_list)
#img_data = img_data.astype('float32')
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print (img_data.shape)

Loaded the images of dataset-dandelion

['14128835667_b6a916222c.jpg', '33522989504_542fde1a1a_n.jpg', '11296320473_1d9261ddcb.jpg', '19621170705_30bf8bf0ba.jpg', '8915661673_9a1cdc3755_m.jpg', '19599413676_fc9ee2640e.jpg', '645330051_06b192b7e1.jpg', '18271576032_d7e2296de4_n.jpg', '16716172029_2166d8717f_m.jpg', '4278757393_bca8415ed4_n.jpg', '15123503538_8ee984abc6.jpg', '15549402199_2890918ddb.jpg', '17146644679_11aff3045c.jpg', '7368449232_c99f49b2e6_n.jpg', '2517777524_e871ec5291_m.jpg', '1193386857_3ae53574f2_m.jpg', '1443259657_2704fab26e_n.jpg', '33851560384_d2e7d933db_n.jpg', '34346200780_c3a783a3ec_n.jpg', '33877161494_05686b7f7a_n.jpg', '34690479536_69da7b98e7_n.jpg', '3844111216_742ea491a0.jpg', '8727612532_6f3d0904aa_n.jpg', '34647026976_68af8875e0_n.jpg', '18970601002_d70bc883a9.jpg', '34662979916_0479576f5e_n.jpg', '2780702427_312333ef33.jpg', '8267315764_129f2e1d77_m.jpg', '9617087594_ec2a9b16f6.jpg', '4552591312_02fe1dcc04_n.jpg', '8684925862_d736e153bf_n.jpg', '25429

UnidentifiedImageError: ignored